<a href="https://colab.research.google.com/github/Aryan-Kamboj11/Multimodal-Hate-Speech-Detection/blob/main/TextPreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch

In [ ]:
main_df = pd.read_csv("/content/drive/MyDrive/Dataset/valid.csv")

In [ ]:
main_df.head()

,text,label,image_path
0,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[1, 0, 0]",Dataset/images/0.jpg
1,“EVERYbody calling you Nigger now!” https://t....,"[1, 0, 1]",Dataset/images/1.jpg
2,@WhiteHouse @realDonaldTrump Fuck ice. White s...,"[5, 1, 1]",Dataset/images/2.jpg
3,#sissy faggot https://t.co/bm1nk8HcYO,"[3, 3, 0]",Dataset/images/3.jpg
4,@Gloriko_ Nigga what? https://t.co/nOwIJtgtU1,"[4, 0, 3]",Dataset/images/4.jpg


In [ ]:
import re
import emoji
import numpy as np

In [ ]:
def remove_urls(text):
  url_pattern = re.compile('https?://\S+|www\.\S+')
  return url_pattern.sub(r'', text)

In [ ]:
def remove_emojis(text):
  demojized_text = emoji.demojize(text)
  emoji_pattern = re.compile(r':\w+:')
  return emoji_pattern.sub('', demojized_text)

In [ ]:
def clean_special_patterns(text):
  text = re.sub(r'@\w+', '[USER]', text)
  text = re.sub(r'#(\w+)', r'\1', text)
  return text.lower()

In [ ]:
def remove_whitespaces(text):
  return re.sub(r'\s+', ' ', text).strip()

In [ ]:
def preprocessingPipeline(df,text_column = "text"):
  if text_column not in df.columns:
        raise ValueError(f"Column '{text_column}' not found in the DataFrame.")

    # Create a working copy to avoid SettingWithCopyWarning
  df_processed = df.copy()

    # Apply all cleaning functions in sequence.
    # We handle non-string data (like NaN) by treating it as an empty string.
  df_processed['cleaned_text'] = df_processed[text_column].fillna('').astype(str)

  print("Step 1: Removing URLs...")
  df_processed['cleaned_text'] = df_processed['cleaned_text'].apply(remove_urls)

  print("Step 2: Removing Emojis...")
  df_processed['cleaned_text'] = df_processed['cleaned_text'].apply(remove_emojis)

  print("Step 3: Cleaning mentions, hashtags, and lowercasing...")
  df_processed['cleaned_text'] = df_processed['cleaned_text'].apply(clean_special_patterns)

  print("Step 4: Normalizing whitespace...")
  df_processed['cleaned_text'] = df_processed['cleaned_text'].apply(remove_whitespaces)

  print("Pipeline finished successfully.")
  return df_processed



In [ ]:
cleaned_df = preprocessingPipeline(main_df)

cleaned_df.head(5)

Step 1: Removing URLs...
Step 2: Removing Emojis...
Step 3: Cleaning mentions, hashtags, and lowercasing...
Step 4: Normalizing whitespace...
Pipeline finished successfully.


,text,label,image_path,cleaned_text
0,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[1, 0, 0]",Dataset/images/0.jpg,rt xxsugvngxx: i ran into this holy nigga today
1,“EVERYbody calling you Nigger now!” https://t....,"[1, 0, 1]",Dataset/images/1.jpg,“everybody calling you nigger now!”
2,@WhiteHouse @realDonaldTrump Fuck ice. White s...,"[5, 1, 1]",Dataset/images/2.jpg,[user] [user] fuck ice. white supremacist tras...
3,#sissy faggot https://t.co/bm1nk8HcYO,"[3, 3, 0]",Dataset/images/3.jpg,sissy faggot
4,@Gloriko_ Nigga what? https://t.co/nOwIJtgtU1,"[4, 0, 3]",Dataset/images/4.jpg,[user] nigga what?


In [ ]:
output_csv_path = "/content/drive/MyDrive/Dataset"

In [ ]:
cleaned_df.to_csv(output_csv_path+"/cleaned_data.csv", index=False)

**TOKENIZATION**

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize_text(text):
  inputs = tokenizer(
      text,
      padding = 'max_length',
      truncation = True,
      max_length = 512,
      return_tensors = 'pt'
  )
  return inputs["input_ids"].squeeze(0),inputs["attention_mask"].squeeze(0)

In [ ]:
cleaned_df["input_ids"],cleaned_df["attention_mask"] = zip(*cleaned_df["text"].apply(tokenize_text))

cleaned_df.head()

,text,label,image_path,cleaned_text,input_ids,attention_mask
0,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[1, 0, 0]",Dataset/images/0.jpg,rt xxsugvngxx: i ran into this holy nigga today,"[tensor(101), tensor(19387), tensor(22038), te...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
1,“EVERYbody calling you Nigger now!” https://t....,"[1, 0, 1]",Dataset/images/1.jpg,“everybody calling you nigger now!”,"[tensor(101), tensor(1523), tensor(7955), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
2,@WhiteHouse @realDonaldTrump Fuck ice. White s...,"[5, 1, 1]",Dataset/images/2.jpg,[user] [user] fuck ice. white supremacist tras...,"[tensor(101), tensor(1030), tensor(2317), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
3,#sissy faggot https://t.co/bm1nk8HcYO,"[3, 3, 0]",Dataset/images/3.jpg,sissy faggot,"[tensor(101), tensor(1001), tensor(24761), ten...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
4,@Gloriko_ Nigga what? https://t.co/nOwIJtgtU1,"[4, 0, 3]",Dataset/images/4.jpg,[user] nigga what?,"[tensor(101), tensor(1030), tensor(1043), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."


In [ ]:
cleaned_df.to_csv(output_csv_path+"/data_tokenized.csv")